In [ ]:
# Utilizando o modelo diretamente fica pesado então foi necesário utilizar o 4b


import torch
from transformers import AutoProcessor, AutoModelForImageTextToText, BitsAndBytesConfig



d:\aaa\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
print(torch.__version__)
print(f"CUDA disponível: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU detectada: {torch.cuda.get_device_name(0)}")

2.6.0+cu124
CUDA disponível: True
GPU detectada: NVIDIA GeForce RTX 3060


In [3]:
model_id = "../Huggingface/hub/models--google--translategemma-4b-it/snapshots/10042cb0e6e7fdce748996a71dc3dc432a4e0c89"
processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForImageTextToText.from_pretrained(model_id, device_map="auto")

The image processor of type `Gemma3ImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 
Loading weights: 100%|██████████| 883/883 [00:42<00:00, 21.00it/s, Materializing param=model.vision_tower.vision_model.post_layernorm.weight]                      


In [17]:
# ---- Text Translation ----
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "source_lang_code": "en",
                "target_lang_code": "pt-BR",
                "text": "He wasn't always this bad, was he? Bayrd thought. He wanted the throne for his wife, but what lord wouldn’t? It was hard to look past the name. Bayrd’s family had followed the Sarand family with reverence for generations",
            }
        ],
    }
]

inputs = processor.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True, return_dict=True, return_tensors="pt"
).to(model.device)

input_len = inputs['input_ids'].shape[1]

with torch.inference_mode():
    generation = model.generate(
        **inputs, 
        do_sample=False,
        max_new_tokens=512, 
        pad_token_id=processor.tokenizer.pad_token_id  
    )

generation = generation[0][input_len:]
decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

Ele não sempre foi assim, não é? Bayrd pensou. Ele queria o trono para sua esposa, mas qual nobre não gostaria? Era difícil ignorar o nome. A família de Bayrd sempre acompanhou a família Sarand com respeito, por gerações.


In [14]:
# ---- Text Extraction and Translation ----
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "source_lang_code": "en",
                "target_lang_code": "pt-BR",
                "url": "https://uploads.tudosaladeaula.com/2024/09/HxOkgb9x-Sem20tC3ADtulo-3179-jpg.webp",
            },
        ],
    }
]

inputs = processor.apply_chat_template(
    messages, tokenize=True, add_generation_prompt=True, return_dict=True, return_tensors="pt"
).to(model.device)

with torch.inference_mode():
    generation = model.generate(**inputs, 
                                do_sample=False,
                                  max_new_tokens=512, 
                                pad_token_id=processor.tokenizer.pad_token_id  )

generation = generation[0][input_len:]
decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)




model
- Oi, Nancy. O que você está fazendo?
Estou enviando um e-mail para um amigo.
- Você usa e-mail com frequência?
- Todos os dias. E você?
- Eu também. Também navego na internet para notícias e esportes.
- A internet é ótima, não é?
- Com certeza. Nancy, você poderia me fazer um favor?
Eu também gostaria de verificar meu e-mail.
- Sem problemas. Darei um jeito em cerca de cinco minutos.
- Obrigado.
